# Train RASCAL ensemble

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
%load_ext line_profiler

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge
from madminer.utils.ml.ratio_trainer import train_ratio_model


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.DEBUG
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Settings

In [6]:
n_estimators = 10

## Training function

In [7]:
def train_ensemble(filename, use_tight_cuts=True, n_estimators=n_estimators, **kwargs):
    cut_label = '_tight' if use_tight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators, debug=False)

    ensemble.train_all(
        method='rascal',
        x_filename=[sample_dir + 'train_ratio{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        theta0_filename=[sample_dir + 'train_ratio{}/theta0_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_ratio{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        r_xz_filename=[sample_dir + 'train_ratio{}/r_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        y_filename=[sample_dir + 'train_ratio{}/y_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.save(model_dir + 'rascal_ensemble_' + filename)

## Line profiler

In [ ]:
%lprun -f train_ratio_model train_ensemble('all_tight', use_tight_cuts=True, n_estimators=1, n_epochs=1, batch_size=1000)


13:23 madminer.ml          INFO    Training 1 estimators in ensemble
13:23 madminer.ml          INFO    Training estimator 1 / 1 in ensemble
13:23 madminer.ml          INFO    Starting training
13:23 madminer.ml          INFO      Method:                 rascal
13:23 madminer.ml          INFO      Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_ratio_tight/x_train_0.npy
13:23 madminer.ml          INFO                     theta0 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_ratio_tight/theta0_train_0.npy
13:23 madminer.ml          INFO                     y at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_ratio_tight/y_train_0.npy
13:23 madminer.ml          INFO                     r_xz at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/train_ratio_tight/r_xz_train_0.npy
13:23 madminer.ml          INFO   

## All observables

In [ ]:
train_ensemble(
    'all',
    use_tight_cuts=False,
    n_epochs=10
)

In [ ]:
train_ensemble(
    'all_tight',
    use_tight_cuts=True,
    n_epochs=1
)

## Just resurrection phi

In [ ]:
train_ensemble(
    'resurrection',
    use_tight_cuts=True,
    features=[[26] for _ in range(n_estimators)],
)